# Apply neural network model to CTD/WOD data
Created by Ivan Lima on Sun Feb 12 2023 10:17:07 -0500

In [1]:
import pandas as pd
import numpy as np
import os, datetime, warnings
print('Last updated on {}'.format(datetime.datetime.now().ctime()))

Last updated on Sun Feb 12 12:45:01 2023


In [2]:
pd.options.display.max_columns = 50
warnings.filterwarnings('ignore')

## Read merged CTD & satellite data

In [3]:
# Set input variables
features = ['Depth', 'Temperature', 'Salinity', 'pCO2_year', 'ADT', 'SST_hires', 'log_KD490']

df_ctd = pd.read_hdf('data/CombinedCTD_satellite_2000-2021.h5', key='df_ctd_sat')
df_ctd['log_KD490'] = np.log(df_ctd.KD490) # log-transformed KD490
df_ctd = df_ctd[~df_ctd[features].isnull().any(axis=1)]

print('Total number of valid CTD points: {:,}\n'.format(len(df_ctd)))
# df_ctd.info()

Total number of valid CTD points: 10,653,902



## Load trained neural network models & apply them to CTD/WOD data

In [4]:
import tensorflow as tf
from tensorflow import keras

model_dic = keras.models.load_model('models/nn_regression_dic_noO2.h5')
model_ta = keras.models.load_model('models/nn_regression_ta_noO2.h5')

X = df_ctd[features].values

# add estimated DIC & TA to dataframe
df_ctd['DIC'] = (model_dic.predict(X)).ravel()
df_ctd['TA'] = (model_ta.predict(X)).ravel()

df_ctd[features + ['DIC', 'TA']].head()

2023-02-12 12:45:16.168553: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


,Depth,Temperature,Salinity,pCO2_year,ADT,SST_hires,log_KD490,DIC,TA
527155,1.984350,8.57,32.401001,373.45,-0.110335,9.777556,-2.493288,2027.638916,2202.627197
527156,2.976518,7.91,32.492001,373.45,-0.110335,9.777556,-2.493288,2039.100586,2209.789795
527157,3.968682,7.74,32.466000,373.45,-0.110335,9.777556,-2.493288,2040.167847,2209.253418
527158,4.960840,7.64,32.556999,373.45,-0.110335,9.777556,-2.493288,2044.105103,2213.754150
527159,5.952994,7.57,32.673000,373.45,-0.110335,9.777556,-2.493288,2048.466064,2218.818359


## Write CTD/WOD data with DIC & TA predictions to HDF5 file 

In [5]:
yr_min, yr_max = df_ctd.Date.dt.year.min(), df_ctd.Date.dt.year.max()
outfile = 'data/CombinedCTD_satellite_bgc_{}-{}.h5'.format(yr_min, yr_max)
print('writing {}'.format(outfile))
df_ctd.to_hdf(outfile, key='df_ctd', mode='w', complevel=9)

writing data/CombinedCTD_satellite_bgc_2002-2018.h5


## Make DIC & TA predictions using 1/4 degree SST instead of high resolution SST 

In [6]:
features2 = ['Depth', 'Temperature', 'Salinity', 'pCO2_year', 'ADT', 'SST', 'log_KD490']

df_ctd = pd.read_hdf('data/CombinedCTD_satellite_2000-2021.h5', key='df_ctd_sat')
df_ctd['log_KD490'] = np.log(df_ctd.KD490) # log-transformed KD490
df_ctd = df_ctd[~df_ctd[features2].isnull().any(axis=1)]

print('Total number of valid CTD points: {:,}\n'.format(len(df_ctd)))

Total number of valid CTD points: 11,178,099



In [7]:
X = df_ctd[features2].values

# add estimated DIC & TA to dataframe
df_ctd['DIC'] = (model_dic.predict(X)).ravel()
df_ctd['TA'] = (model_ta.predict(X)).ravel()

df_ctd[features2 + ['DIC', 'TA']].head()

,Depth,Temperature,Salinity,pCO2_year,ADT,SST,log_KD490,DIC,TA
0,1.487919,3.561,29.858999,369.71,-0.118689,4.510375,-2.786163,1987.524414,2164.142334
1,1.983890,3.565,31.264999,369.71,-0.118689,4.510375,-2.786163,2031.810059,2192.474854
2,2.479860,3.569,30.969000,369.71,-0.118689,4.510375,-2.786163,2021.571533,2184.806885
3,2.975828,3.567,30.922001,369.71,-0.118689,4.510375,-2.786163,2019.959839,2183.809814
4,3.471795,3.568,30.922001,369.71,-0.118689,4.510375,-2.786163,2020.064331,2183.827393


In [8]:
yr_min, yr_max = df_ctd.Date.dt.year.min(), df_ctd.Date.dt.year.max()
outfile = 'data/CombinedCTD_satellite_bgc_{}-{}.h5'.format(yr_min, yr_max)
print('writing {}'.format(outfile))
df_ctd.to_hdf(outfile, key='df_ctd', mode='w', complevel=9)

writing data/CombinedCTD_satellite_bgc_2000-2018.h5
